#Deep MNIST para Expertos

TensorFlow es una poderosa libreria para hacer calculos numericos a gran escala. Una de las tareas que realiza esta libreria es implemntar y entrenar deep neural networks. En este tutorial se aprende los bloques basicos para construir un modelo TensorFlow mientras se construye un clasificador deep convolutional MNIST.

#Preparacion

Antes de crear el modelo primero se debe cargar el dataset MNIST y despues iniciar una sesion TensorFlow.

#Cargar la informacion de MNIST

Para cargar la informacion se incluye un script que automaticamente descarga e importa el dataset de MNIST. Cuando se ejecuta este script se crea un directorio con en nombre de 'MNIST_data' en el cual se guardan los archivos con los datos.

a continuacion se presentan las lineas que ejecutan dicho script:

In [1]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


La variable mnist contiene los arreglos de tipo numpy de entrenamiento, validacion y testing. Ademas provee una funcion que permite iterar atraves de los minibatches de datos los cuales se utilizan mas adelante.

#Comenzar una sesion interactiva en TensorFlow

TensorFlow se basa en un eficiente backend de C++ para hacer los calculos. La conexion para este backend se llama sesion. El uso mas comun para programas TensorFlow es primero crear un grafo y despues iniciar una sesion.

Aqui se utiliza por conveniciencia la clase InteractiveSession la cual hace que TensorFlow sea aun mas flexible acerca de la estructura del codigo Tambien permite intercalar operaciones las cuales construyen un grafo de calculos con el cual se ejecutara el grafo. Esto es particularmente conveniente cuando se trabaja con contextos interactivos como iPython. 

A continuacion se inicia la sesion:

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

#Computacion Grafica (Cambiar este nombre!)

Para realizar calculos numericos eficientes en python usualmente se utilizan librerias como NumPy que hacen operaciones costosas como las multiplicacion de matrices fuera de Python utilizando codigo eficiente implementado en otro lenguaje. Desafortunadamente existe una gran cantidad de overhead al cambiar de una operacion a otra.

TensorFlow utiliza librerias implementadas en otros lenguajes pero evitando el overhead. Provee un grafo con operaciones interactivas las cuales corren fuera de python.

El rol de codigo en python es construir un grafo externo y dictarle que partes de el grafo se van a utilizar.

# Construir el Modelo de la Regresion Softmax

En esta seccion se construye el modelo de la regresion softmax con una sola capa. En la siguiente seccion se extiene el caso de la regresion softmax con multicapas en una convolutional network.